<a href="https://colab.research.google.com/github/Hojiakbardevs/Datascience/blob/main/regression_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import os
import math
import numpy as np
import pandas as pd
import seaborn as sns
from IPython.display import display

#from brokenaxes import brokenaxes
from statsmodels.formula import api
from sklearn.feature_selection import RFE
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.decomposition import PCA
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10,6]

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# walmart uchun data malumotlarni regression algoritmlar yordamida o'rganib olish

In [9]:
# prompt: how can I import csv folder in github at the colab

!git clone 'https://github.com/Hojiakbardevs/data-for-datascience.git'

fatal: destination path 'data-for-datascience' already exists and is not an empty directory.


In [23]:
df = pd.read_csv('/content/data-for-datascience/Walmart.csv')
df.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,1,05-02-2010,1643690.90,0,42.31,2.572,211.096358,8.106
1,1,12-02-2010,1641957.44,1,38.51,2.548,211.242170,8.106
2,1,19-02-2010,1611968.17,0,39.93,2.514,211.289143,8.106
3,1,26-02-2010,1409727.59,0,46.63,2.561,211.319643,8.106
4,1,05-03-2010,1554806.68,0,46.50,2.625,211.350143,8.106


In [24]:
display(df.head())

original_df = df.copy(deep=True)

print('\n\033[1mInference:\033[0m The Datset consists of {} features & {} samples.'.format(df.shape[1], df.shape[0]))

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,1,05-02-2010,1643690.90,0,42.31,2.572,211.096358,8.106
1,1,12-02-2010,1641957.44,1,38.51,2.548,211.242170,8.106
2,1,19-02-2010,1611968.17,0,39.93,2.514,211.289143,8.106
3,1,26-02-2010,1409727.59,0,46.63,2.561,211.319643,8.106
4,1,05-03-2010,1554806.68,0,46.50,2.625,211.350143,8.106



Inference: The Datset consists of 8 features & 6435 samples.


In [25]:
df = pd.DataFrame(df)
df.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,1,05-02-2010,1643690.90,0,42.31,2.572,211.096358,8.106
1,1,12-02-2010,1641957.44,1,38.51,2.548,211.242170,8.106
2,1,19-02-2010,1611968.17,0,39.93,2.514,211.289143,8.106
3,1,26-02-2010,1409727.59,0,46.63,2.561,211.319643,8.106
4,1,05-03-2010,1554806.68,0,46.50,2.625,211.350143,8.106


In [27]:
df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y') # Changed the date format to day-month-year

# Extract weekday, month, and year
df['weekday'] = df['Date'].dt.weekday
df['month'] = df['Date'].dt.month
df['year'] = df['Date'].dt.year

# Map months to quarters
df['Monthly_Quarter'] = df['month'].map({
    1: 'Q1', 2: 'Q1', 3: 'Q1', 4: 'Q2', 5: 'Q2', 6: 'Q2',
    7: 'Q3', 8: 'Q3', 9: 'Q3', 10: 'Q4', 11: 'Q4', 12: 'Q4'
})

# Drop the original Date column
df.drop(['Date'], axis=1, inplace=True)

,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,weekday,month,year,Monthly_Quarter
0,1,1643690.90,0,42.31,2.572,211.096358,8.106,4,2,2010,Q1
1,1,1641957.44,1,38.51,2.548,211.242170,8.106,4,2,2010,Q1
2,1,1611968.17,0,39.93,2.514,211.289143,8.106,4,2,2010,Q1
3,1,1409727.59,0,46.63,2.561,211.319643,8.106,4,2,2010,Q1
4,1,1554806.68,0,46.50,2.625,211.350143,8.106,4,3,2010,Q1


,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,weekday,month,year,Monthly_Quarter
213,2,1929153.16,0,83.40,3.648,214.696491,7.931,4,6,2011,Q2
3683,26,955641.74,0,22.49,3.827,137.423897,7.467,4,3,2012,Q1
4001,28,1205536.71,0,71.74,4.468,131.108333,10.199,4,10,2012,Q4
235,2,1920725.15,0,59.33,3.297,217.645539,7.441,4,11,2011,Q4
6228,44,290399.66,0,75.95,3.606,129.201581,6.560,4,8,2011,Q3
148,2,1990483.78,0,57.56,2.667,211.037551,8.324,4,3,2010,Q1
